In [1]:
import os
from keras.models import load_model
import pandas as pd

Using TensorFlow backend.


In [2]:
GPU = '1'
MODEL_FILE = '__run_c_e24.h5'
INPUT_DIR = '/d2/caches/tf-speech/test/audio'
OUT_DIR = 'out'
OUT_CSV_FILE = OUT_DIR + '/predictions-a.csv'

In [3]:
%run 'lib.ipynb'
%run 'data-generator.ipynb'

In [4]:
# make only specific GPU to be utilized
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = GPU

In [5]:
test_files_file = '%s/test_files.npy' % OUT_DIR
test_X_file = '%s/test_X.mem' % OUT_DIR

In [6]:
dg = DataGenerator(INPUT_DIR, extra_silence_dir=None)

In [7]:
model = load_model(MODEL_FILE)

In [8]:
if os.path.isfile(test_files_file):
    test_files = np.load(test_files_file)
else:
    test_files = dg.input_files[None]
    np.save(test_files_file, test_files)
    
print('test files: %d' % len(test_files))

test files: 158538


In [ ]:
# # generate msgs for test files
# if not os.path.isfile(test_X_file):

#     if os.path.isfile(test_X_file + '.tmp'):
#         os.unlink(test_X_file + '.tmp')

#     dg.debug = False
#     test_X = np.memmap(
#         test_X_file + '.tmp',
#         np.float32,
#         'w+',
#         shape=(len(test_files), dg.n_mels, dg.msg_w, 1))

#     for i, file in tqdm(enumerate(test_files)):
#         wave, _ = dg.generate_audio(label='-', file=file, transform=False)
#         msg = dg.msg(wave)
#         msg = dg.normalize_msg(msg)
#         test_X[i] = msg[:, :, np.newaxis]

#     test_X.flush()
#     os.rename(src=test_X_file + '.tmp', dst=test_X_file)

# test_X = np.memmap(
#     test_X_file,
#     np.float32,
#     'r',
#     shape=(len(test_files), dg.n_mels, dg.msg_w, 1))

# assert len(test_files) == len(test_X)
# print('test files: %d' % len(test_X))

In [ ]:
predictions = {}

# dynamically predict

for file in tqdm(test_files):
    wave, _ = dg.generate_audio(label='-', file=file, transform=False)
    msg = dg.msg(wave)
    msg = dg.normalize_msg(msg)

    p = model.predict(msg[np.newaxis, :, :, np.newaxis])
    predictions[file] = p[0]

predictions_pd = pd.DataFrame.from_dict(predictions, orient='index')
predictions_pd.to_csv(OUT_CSV_FILE, index_label='file')

  0%|          | 514/158538 [00:34<2:57:23, 14.85it/s]